Note: Choose one of the two cells to run, and replace the file location with your own file location

In [1]:
# For glb file with objects grouped by inserts; assumes hierarchy similar to fullassebly.dwg
import ezdxf
from ezdxf.acis import api as acis
from pygltflib import (
    GLTF2, Scene, Node, Mesh, Primitive, Accessor, Buffer, BufferView, Attributes,
    FLOAT, UNSIGNED_INT, UNSIGNED_SHORT, TRIANGLES
)
import numpy as np
from ezdxf.math import Matrix44, Vec3

# Replace with the filepath of your dxf file
try:
    doc = ezdxf.readfile("fullassebly.dxf")
except FileNotFoundError:
    print("Error: DXF file not found.")
    exit()

msp = doc.modelspace()

gltf = GLTF2()
scene = Scene()
gltf.scenes.append(scene)
gltf.scene = 0

all_named_meshes = []
all_binary_data = []
current_binary_offset = 0

def process_acis_entity(entity):
    meshes = []
    try:
        bodies = acis.load_dxf(entity)
        for body in bodies:
            for mesh in acis.mesh_from_body(body, merge_lumps=True):
                vertices = mesh.vertices
                faces = mesh.faces
                meshes.append({"vertices": vertices, "faces": faces})
    except Exception as e:
        print(f"ACIS error on {entity.dxf.handle}: {e}")
    return meshes

def process_block_geometry(block, current_transform, mesh_list, parent_name="", parent_node_index=None):
    group_node = Node(
        name=parent_name,
        children=[]
    )
    gltf.nodes.append(group_node)
    this_node_index = len(gltf.nodes) - 1

    if parent_node_index is not None:
        gltf.nodes[parent_node_index].children.append(this_node_index)
    else:
        gltf.scenes[0].nodes.append(this_node_index)

    for entity in block.query('3DSOLID'):
        solid_meshes = process_acis_entity(entity)
        for i, mesh_data in enumerate(solid_meshes):
            transformed_vertices = [
                current_transform.transform(vertex) for vertex in mesh_data["vertices"]
            ]
            mesh_name = f"{parent_name}_Solid_{entity.dxf.handle}_{i}"
            mesh_list.append({
                "vertices": transformed_vertices,
                "faces": mesh_data["faces"],
                "name": mesh_name,
                "parent_node_index": this_node_index
            })

    for insert in block.query('INSERT'):
        local_transform = insert.matrix44()
        combined_transform = local_transform @ current_transform
        nested_block = doc.blocks.get(insert.dxf.name)
        if nested_block:
            nested_name = f"{insert.dxf.name}_{insert.dxf.handle}"
            process_block_geometry(
                nested_block,
                combined_transform,
                mesh_list,
                parent_name=nested_name,
                parent_node_index=this_node_index
            )

for insert in msp.query('INSERT'):
    block = doc.blocks.get(insert.dxf.name)
    if block:
        transform = insert.matrix44()
        name = f"{insert.dxf.name}_{insert.dxf.handle}"
        process_block_geometry(block, transform, all_named_meshes, parent_name=name)

identity = Matrix44()
for entity in msp.query('3DSOLID'):
    model_meshes = process_acis_entity(entity)
    for i, mesh_data in enumerate(model_meshes):
        transformed_vertices = [
            identity.transform(vertex) for vertex in mesh_data["vertices"]
        ]
        mesh_name = f"ModelspaceSolid_{entity.dxf.handle}_{i}"
        all_named_meshes.append({
            "vertices": transformed_vertices,
            "faces": mesh_data["faces"],
            "name": mesh_name,
            "parent_node_index": None
        })

for mesh_data in all_named_meshes:
    vertices = np.array(mesh_data["vertices"], dtype=np.float32)
    faces = mesh_data["faces"]
    name = mesh_data["name"]
    parent_idx = mesh_data.get("parent_node_index")

    triangulated_faces = []
    for face in faces:
        if len(face) == 3:
            triangulated_faces.append(face)
        elif len(face) == 4:
            triangulated_faces.append([face[0], face[1], face[2]])
            triangulated_faces.append([face[0], face[2], face[3]])
        elif len(face) > 4:
            for i in range(1, len(face) - 1):
                triangulated_faces.append([face[0], face[i], face[i+1]])
        else:
            continue

    if not triangulated_faces:
        print(f"Skipping mesh '{name}': no valid faces")
        continue

    faces_dtype = np.uint32 if len(vertices) > 65535 else np.uint16
    index_component_type = UNSIGNED_INT if faces_dtype == np.uint32 else UNSIGNED_SHORT

    faces_array = np.array(triangulated_faces, dtype = faces_dtype)
    faces_flat = faces_array.flatten()

    all_binary_data.append(vertices.tobytes())
    all_binary_data.append(faces_flat.tobytes())

    v_buf = BufferView(buffer = 0, byteOffset = current_binary_offset, byteLength = vertices.nbytes)
    gltf.bufferViews.append(v_buf)
    current_binary_offset += vertices.nbytes

    i_buf = BufferView(buffer = 0, byteOffset = current_binary_offset, byteLength = faces_flat.nbytes)
    gltf.bufferViews.append(i_buf)
    current_binary_offset += faces_flat.nbytes

    v_accessor = Accessor(
        bufferView = len(gltf.bufferViews) - 2,
        byteOffset = 0,
        componentType = FLOAT,
        count = len(vertices),
        type = "VEC3",
        min = vertices.min(axis=0).tolist(),
        max = vertices.max(axis=0).tolist()
    )
    i_accessor = Accessor(
        bufferView = len(gltf.bufferViews) - 1,
        byteOffset = 0,
        componentType = index_component_type,
        count = len(faces_flat),
        type = "SCALAR"
    )
    gltf.accessors.extend([v_accessor, i_accessor])

    primitive = Primitive(
        attributes = Attributes(POSITION = len(gltf.accessors) - 2),
        indices = len(gltf.accessors) - 1,
        mode = TRIANGLES
    )
    mesh = Mesh(primitives = [primitive])
    gltf.meshes.append(mesh)

    node = Node(name = name, mesh = len(gltf.meshes) - 1)
    gltf.nodes.append(node)
    node_index = len(gltf.nodes) - 1

    if parent_idx is not None:
        gltf.nodes[parent_idx].children.append(node_index)
    else:
        gltf.scenes[0].nodes.append(node_index)

gltf.buffers.append(Buffer(byteLength = len(b''.join(all_binary_data))))
gltf.set_binary_blob(b''.join(all_binary_data))
gltf.save("output.glb")

print("GLB file created: output.glb")


GLB file created: output.glb


In [2]:
# For glb file with all individual panel objects
import ezdxf
from ezdxf.acis import api as acis
from pygltflib import (
    GLTF2, Scene, Node, Mesh, Primitive, Accessor, Buffer, BufferView, Attributes,
    FLOAT, UNSIGNED_INT, UNSIGNED_SHORT, TRIANGLES
)
import numpy as np
from ezdxf.math import Matrix44, Vec3

# Replace with the filepath of your dxf file
try:
    doc = ezdxf.readfile("fullassebly.dxf")
except FileNotFoundError:
    print("Error: DXF file not found. Please ensure the file is in the same directory or provide the correct path.")
    exit()

msp = doc.modelspace()

all_named_meshes = []

def process_acis_entity(entity):
    meshes = []
    try:
        bodies = acis.load_dxf(entity)
        for body in bodies:
            for mesh in acis.mesh_from_body(body, merge_lumps=True):
                vertices = mesh.vertices
                faces = mesh.faces
                meshes.append({"vertices": vertices, "faces": faces})
    except Exception as e:
        print(f"Could not load/mesh ACIS data from {entity.dxftype()} {entity.dxf.handle}: {e}")
    return meshes

def process_block_geometry(block, current_cumulative_transform, named_meshes_list, parent_name=""):
    for entity in block.query('3DSOLID'):
        block_meshes = process_acis_entity(entity)
        for i, mesh_data in enumerate(block_meshes):
            transformed_vertices = []
            for vertex in mesh_data["vertices"]:
                transformed_vertex = current_cumulative_transform.transform(vertex)
                transformed_vertices.append(transformed_vertex)
            
            mesh_name = f"{parent_name}_Solid_{entity.dxf.handle}_{i}" if parent_name else f"Solid_{entity.dxf.handle}_{i}"
            named_meshes_list.append({
                "vertices": transformed_vertices,
                "faces": mesh_data["faces"],
                "name": mesh_name
            })

    for insert in block.query('INSERT'):
        local_insert_transform = insert.matrix44()
        cumulative_nested_transform = local_insert_transform @ current_cumulative_transform

        nested_block_name = insert.dxf.name
        nested_block = doc.blocks.get(nested_block_name)
        if nested_block:
            process_block_geometry(nested_block, cumulative_nested_transform, named_meshes_list, parent_name=nested_block_name)


# Main Processing Loop

for insert in msp.query('INSERT'):
    block_name = insert.dxf.name
    block = doc.blocks.get(block_name)
    if block:
        initial_transform = insert.matrix44()
        process_block_geometry(block, initial_transform, all_named_meshes, parent_name=block_name)

for entity in msp.query('3DSOLID'):
    modelspace_meshes = process_acis_entity(entity)
    identity_matrix = Matrix44()
    for i, mesh_data in enumerate(modelspace_meshes):
        transformed_vertices = []
        for vertex in mesh_data["vertices"]:
            transformed_vertex = identity_matrix.transform(vertex)
            transformed_vertices.append(transformed_vertex)
        
        mesh_name = f"ModelspaceSolid_{entity.dxf.handle}_{i}"
        all_named_meshes.append({
            "vertices": transformed_vertices,
            "faces": mesh_data["faces"],
            "name": mesh_name
        })
        
gltf = GLTF2()
scene = Scene()
gltf.scenes.append(scene)
gltf.scene = 0

all_binary_data = [] 

current_binary_offset = 0

for mesh_data in all_named_meshes:
    vertices = np.array(mesh_data["vertices"], dtype=np.float32)
    original_faces = mesh_data["faces"]
    mesh_name = mesh_data["name"]

    triangulated_faces = []
    for face in original_faces:
        num_vertices = len(face)
        if num_vertices == 3:
            triangulated_faces.append(list(face))
        elif num_vertices == 4:
            triangulated_faces.append([face[0], face[1], face[2]])
            triangulated_faces.append([face[0], face[2], face[3]])
        elif num_vertices > 4:
            for i in range(1, num_vertices - 1):
                triangulated_faces.append([face[0], face[i], face[i + 1]])
        else:
            print(f"Warning: Skipping face with {num_vertices} vertices for mesh '{mesh_name}'.")
            continue

    if not triangulated_faces:
        print(f"Warning: No valid triangulated faces found for mesh '{mesh_name}'. Skipping GLB mesh creation.")
        continue

    if len(vertices) > 65535:
        faces_dtype = np.uint32
        index_component_type = UNSIGNED_INT
    else:
        faces_dtype = np.uint16
        index_component_type = UNSIGNED_SHORT

    faces_array = np.array(triangulated_faces, dtype=faces_dtype)
    faces_flattened = faces_array.flatten()

    all_binary_data.append(vertices.tobytes())
    all_binary_data.append(faces_flattened.tobytes())

    # Vertices BufferView
    vertex_buffer_view = BufferView()
    vertex_buffer_view.buffer = 0
    vertex_buffer_view.byteOffset = current_binary_offset
    vertex_buffer_view.byteLength = vertices.nbytes
    gltf.bufferViews.append(vertex_buffer_view)
    current_binary_offset += vertices.nbytes

    # Indices BufferView
    index_buffer_view = BufferView()
    index_buffer_view.buffer = 0
    index_buffer_view.byteOffset = current_binary_offset
    index_buffer_view.byteLength = faces_flattened.nbytes
    gltf.bufferViews.append(index_buffer_view)
    current_binary_offset += faces_flattened.nbytes

    # Accessors
    position_accessor = Accessor()
    position_accessor.bufferView = len(gltf.bufferViews) - 2
    position_accessor.byteOffset = 0
    position_accessor.componentType = FLOAT
    position_accessor.count = len(vertices)
    position_accessor.type = "VEC3"
    position_accessor.min = vertices.min(axis=0).tolist()
    position_accessor.max = vertices.max(axis=0).tolist()
    gltf.accessors.append(position_accessor)

    indices_accessor = Accessor()
    indices_accessor.bufferView = len(gltf.bufferViews) - 1
    indices_accessor.byteOffset = 0
    indices_accessor.componentType = index_component_type
    indices_accessor.count = len(faces_flattened)
    indices_accessor.type = "SCALAR"
    gltf.accessors.append(indices_accessor)

    # Primitive and mesh
    primitive = Primitive(mode=TRIANGLES)
    primitive.attributes = Attributes(POSITION=len(gltf.accessors) - 2)
    primitive.indices = len(gltf.accessors) - 1

    mesh = Mesh()
    mesh.primitives.append(primitive)
    gltf.meshes.append(mesh)

    # Node for the mesh
    node = Node()
    node.mesh = len(gltf.meshes) - 1
    node.name = mesh_name
    gltf.nodes.append(node)
    gltf.scenes[0].nodes.append(len(gltf.nodes) - 1) # Add node to the scene's root nodes

# Create a single Buffer for all binary data
combined_binary_data = b''.join(all_binary_data)
main_buffer = Buffer()
main_buffer.byteLength = len(combined_binary_data)
gltf.buffers.append(main_buffer)
gltf.set_binary_blob(combined_binary_data)

gltf.save("output.glb")

print("GLB file created: output.glb")


GLB file created: output.glb
